## The DF generated after crawling the NationMaster site was not used eventually due to duplicated data.

In [2]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
#import scipy as sc
#import numpy as np
import time
import os

In [3]:
def load_soup_object(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    #soup = BeautifulSoup(plain_text)
    soup = BeautifulSoup(plain_text,features="lxml") # added...
    return soup

In [4]:
def save_csv(dataframe, csv_name, folder_name):
    csv_path = folder_name + os.sep + csv_name
    dataframe.to_csv(csv_path)

In [5]:
def divide_and_save_to_df(country_name, mixed_data):
    num = mixed_data.__len__()
    year = list()
    consumption = list()
    for x in range(1, num, 2):
        year.append(mixed_data[x-1].strip())
        consumption.append(mixed_data[x].strip())
    df = pd.DataFrame({"Country": country_name, "Year": year, "Consumption": consumption})
    return df

In [6]:
base_url='https://www.nationmaster.com'
url = base_url + '/nmx/sector/alcoholic-drink'
soup = load_soup_object(url)

In [7]:
# find a category list for alcoholic-drinks .
drink_link = list()
soup = soup('a', attrs={'data-event-action': 'Click Sector'})
for obj in soup:
    if not obj.text.__contains__('Cider'):
        drink_link.append(obj['href'])

time.sleep(0.5)

In [8]:
country_link = list()
country_name = list()
year_and_value = list()

for page in drink_link:
    url = base_url + page
    soup = load_soup_object(url)
    soup = soup('a', attrs={'data-event-action': 'Click Ranking 1'})
    for obj in soup:
        if obj.text.strip().__contains__('Consumption'):
            link = obj['href']
    url = base_url + link
    time.sleep(0.5)
    soup = load_soup_object(url)
    soup = soup('a', attrs={'class':'cta cta-series-data'})
    for obj in soup: # country_link of links.
        country_link.append(obj['href'])

    for country in country_link:
        time.sleep(0.5)
        url = base_url + country
        soup = load_soup_object(url)
        name_soup = soup('h1') # for country name.
        country_name += 5 * [name_soup[0].text.strip()]
        table_soup = soup('table')
        table_td = table_soup[0].findAll('td')
        for td in table_td:
            year_and_value.append(td.text.strip())

In [9]:
df = divide_and_save_to_df(country_name, year_and_value) 
save_csv(df, 'nation.csv', 'nationmaster')